<a href="https://colab.research.google.com/github/shriram9992/anomaly_detection/blob/master/lof_shuttle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving shuttle.csv to shuttle.csv


In [0]:
import pandas as pd
import io
df2 = pd.read_csv(io.BytesIO(uploaded['shuttle.csv']),header=None)

In [39]:
nor1 = df2.loc[df2[9]==1]
nor2 = df2.loc[df2[9]==4]
nor = nor1.append(nor2)
nor.reset_index(inplace = True,drop = True)
print(nor.head())

    0  1    2  3   4   5   6   7  8  9
0  53  0   82  0  52  -5  29  30  2  1
1  37  0   76  0  28  18  40  48  8  1
2  37  0   79  0  34 -26  43  46  2  1
3  55 -1   95 -3  54  -4  40  41  2  1
4  37  0  101 -7  28   0  64  73  8  1


In [40]:
ano1 = df2.loc[df2[9]==2]
ano2 = df2.loc[df2[9]==3]
ano3 = df2.loc[df2[9]==5]
ano = ano1.append(ano2)
ano = ano.append(ano3)
ano.reset_index(inplace = True,drop = True)
print(ano.head())

    0   1   2  3   4  5   6    7    8  9
0  50  21  77  0  28  0  27   48   22  2
1  70  57  82  0 -42 -4  12  126  114  2
2  42  36  77  0 -22  0  36  101   64  2
3  69  59  82  0 -42 -5  13  126  112  2
4  68  67  82  0 -42 -6  13  126  112  2


In [41]:
import numpy as np
nor.apply(np.random.shuffle,axis=0)
print(nor.head())

    0  1   2  3   4   5   6   7   8  9
0  41  4  95  0  50   2  47  52  28  1
1  53  3  76  0  26  16  31  32  16  1
2  43  0  96  0  46  11  60  58  18  1
3  37  0  85  0  52   0  32  87  76  1
4  37  0  80  0  34  -3  43  43   2  1


In [42]:
ano.apply(np.random.shuffle,axis=0)
print(ano.head())

     0   1    2  3   4   5  6    7    8  9
0   78   3   86  1  72  -4  5   95   92  5
1  102   0   88  0  -4  -3  4  100  120  5
2  106  -3   84 -3 -22 -16  4   37   86  5
3  104  21  104  3  70 -13  5   88   80  5
4   71   0  102  6  72   6  4  121  104  5


In [44]:
X_train = nor.loc[0:43592,:]
X_train = X_train.drop(9,1)
X_train.reset_index(inplace = True,drop = True)
print(X_train.head())

    0  1   2  3   4   5   6   7   8
0  41  4  95  0  50   2  47  52  28
1  53  3  76  0  26  16  31  32  16
2  43  0  96  0  46  11  60  58  18
3  37  0  85  0  52   0  32  87  76
4  37  0  80  0  34  -3  43  43   2


In [45]:
X_test = nor.loc[43592:54489,:]
X_test1 = ano.loc[:,:]
X_test = X_test.append(X_test1)
X_test.reset_index(inplace = True,drop = True)
X_test = X_test.drop(9,1)
print(X_test.head())

    0  1    2  3   4   5   6   7   8
0  37  0   86  3  42   0  43  43   0
1  39  0   83  0  -4   0  40  29   8
2  46  4   82  0  52   0  54  86   0
3  46  0  102 -1  10   3  40  39   2
4  43  0   97  0  34  13  42  39  46


In [46]:
y_test1 = nor.loc[43592:54489,9]
y_test2 = ano.loc[:,9]
y_test = y_test1.append(y_test2)
y_test.reset_index(inplace = True,drop = True)
y_test = y_test.map({1:1,4:1,2:-1,3:-1,5:-1})
print(y_test.head())


0    1
1    1
2    1
3    1
4    1
Name: 9, dtype: int64


In [0]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [0]:
def accuracy(y_test,y_pred):
  TP = FN = FP = TN = 0
  for j in range(len(y_test)):
      if y_test[j]== 1 and y_pred[j] == 1:
          TN = TN+1
      elif y_test[j]== 1 and y_pred[j] == -1:
          FP = FP+1
      elif y_test[j]== -1 and y_pred[j] == 1:
          FN = FN+1
      else:
          TP = TP +1
  precision = TP/(TP+FP)
  recall = TP/(TP+FN)
  return (precision,recall)

In [49]:
c = [0.1,0.01,0.001]
n = [10,20,30]
for i in c:
  for j in n:
    from sklearn.neighbors import LocalOutlierFactor
    classifier = LocalOutlierFactor(n_neighbors=j, contamination=i, novelty=True)
    classifier.fit(X_train)
    y_pred = classifier.predict(X_test)
    (pre,rec)=accuracy(y_test,y_pred)
    print(i,j)
    print(pre,rec)
    


0.1 10
0.7464155788572652 1.0
0.1 20
0.7538361789496434 1.0
0.1 30
0.7515621633268692 1.0
0.01 10
0.9686023895526535 0.9994266055045872
0.01 20
0.971850613154961 0.9997133027522935
0.01 30
0.9713091922005571 0.9997133027522935
0.001 10
0.9907949790794979 0.3394495412844037
0.001 20
0.9137931034482759 0.0455848623853211
0.001 30
0.7796610169491526 0.013188073394495414
